# Consulta 05

Esta consulta devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de rese;as de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo, en caso de no estar categorizados o encontrarse arroja el mensaje "No se encontró información sobre el desarrollador '...'".

Endpoint: def developer_reviews_analysis( desarrolladora : str )\
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}



*Importamos la librería necesaria para el procesamiento de los datos*

In [4]:
import pandas as pd


*Importamos los archivos con los que vamos a trabajar la consulta*

In [5]:
df_games = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_games_clean.parquet')
sentiment = pd.read_csv(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\Sentiment_Analysis_csv.unknown')

*Realizamos una copia para trabajar sobre ella y mantener los datos intactos en el archivo original.*

In [6]:
games_copy = df_games.copy()
sentiment_copy = sentiment.copy()

*Desarrollamos la función correspondiente para hacer la consulta que necesitamos*

In [7]:
def developer_reviews_analysis(desarrollador):
    # Combinar conjuntos de datos en las columnas apropiadas ('item_id' en reviews y 'id' en games)
    merged_data = pd.merge(sentiment_copy, games_copy, left_on='item_id', right_on='id')
    
    # Filtrar filas donde el puntaje de sentimiento es positivo (2) o negativo (0)
    filtered_data = merged_data[merged_data['sentiment_analysis'] != 1]  # Excluir sentimiento neutral
    
    # Agrupar por desarrollador y puntaje de sentimiento, contar la cantidad de resenas
    grouped_data = filtered_data.groupby(['developer', 'sentiment_analysis']).size().unstack(fill_value=0)
    
    # Verificar si el desarrollador está en el DataFrame para manejar excepciones
    if desarrollador in grouped_data:
        # Extraer cantidad de resenas positivas y negativas para la desarrollador especificada
        developer_reviews = grouped_data.loc[desarrollador]
    
        # Convertir cantidades a formato de lista con claves especificadas
        developer_reviews_list = [
            {"Negativas": developer_reviews.get(0, 0)},
            {"Positivas": developer_reviews.get(2, 0)}
        ]
    
        return {desarrollador: developer_reviews_list}
    else:
        return f"No se encontró información sobre el desarrollador {desarrollador}"


*Aplicamos la función y corroboramos que funcione correctamente*

In [8]:
# Example usage:
developer = input("Ingrese nombre del desarrollador: ")
resultado = developer_reviews_analysis(developer)
print(resultado)

ValueError: You are trying to merge on float64 and object columns for key 'item_id'. If you wish to proceed you should use pd.concat